In [26]:
import pandas as pd
import gzip

In [27]:
# load_rating_df

rating_df = pd.read_csv('../data/beauty/ratings_Beauty.csv')
rating_df.columns = ['uid', 'sid', 'rating', 'timestamp']
rating_df.head()

,uid,sid,rating,timestamp
0,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
1,A1Z513UWSAAO0F,0558925278,5.0,1404691200
2,A1WMRR494NWEWV,0733001998,4.0,1382572800
3,A3IAAVS479H7M7,0737104473,1.0,1274227200
4,AKJHHD5VEH7VG,0762451459,5.0,1404518400


In [28]:
# load_metadict

path = "../data/beauty/meta_Beauty.json.gz"
meta_dict = {}

with gzip.open(path, 'rb') as f:
    for line in f:
        item = eval(line)
        if 'title' in item and len(item['title']) > 0:
            meta_dict[item['asin'].strip()] = item['title'].strip()

# 5개 출력
for i in list(meta_dict.keys())[:5]:
    print(f"{i} : '{meta_dict[i]}'")


0205616461 : 'Bio-Active Anti-Aging Serum (Firming Ultra-Hydrating Serum)'
0558925278 : 'Eco Friendly Ecotools Quality Natural Bamboo Cosmetic Mineral Brush Set Kit of 4 Soft Brushes and 1 Pouch Baby Kabuki Eye Shading Brush Mineral Powder Brush Concealer Brush(travle Size)'
0733001998 : 'Mastiha Body Lotion'
0737104473 : 'Hello Kitty Lustre Lipstick (See sellers comments for colors)'
0762451459 : 'Stephanie Johnson Mermaid Round Snap Mirror'


In [29]:
df = rating_df.copy()
meta_raw = meta_dict
df = df[df['sid'].isin(meta_raw)]
df.head()

,uid,sid,rating,timestamp
0,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
1,A1Z513UWSAAO0F,0558925278,5.0,1404691200
2,A1WMRR494NWEWV,0733001998,4.0,1382572800
3,A3IAAVS479H7M7,0737104473,1.0,1274227200
4,AKJHHD5VEH7VG,0762451459,5.0,1404518400


min_sc 과 min_uc는 하이퍼파라미터이다. 평점 개수가 적은 아이템은 제외하기 위한 용도로 보임.

In [30]:
def filter_triplets(self, df):
    print('Filtering triplets')
    if self.min_sc > 1 or self.min_uc > 1:
        item_sizes = df.groupby('sid').size()
        good_items = item_sizes.index[item_sizes >= self.min_sc]
        user_sizes = df.groupby('uid').size()
        good_users = user_sizes.index[user_sizes >= self.min_uc]
        while len(good_items) < len(item_sizes) or len(good_users) < len(user_sizes):
            if self.min_sc > 1:
                item_sizes = df.groupby('sid').size()
                good_items = item_sizes.index[item_sizes >= self.min_sc]
                df = df[df['sid'].isin(good_items)]

            if self.min_uc > 1:
                user_sizes = df.groupby('uid').size()
                good_users = user_sizes.index[user_sizes >= self.min_uc]
                df = df[df['uid'].isin(good_users)]

            item_sizes = df.groupby('sid').size()
            good_items = item_sizes.index[item_sizes >= self.min_sc]
            user_sizes = df.groupby('uid').size()
            good_users = user_sizes.index[user_sizes >= self.min_uc]
    return df

In [31]:
min_sc = 2
min_uc = 2

In [32]:
# 내역이 하이퍼파라미터 개수보다 적은 항목은 제거
item_sizes = df.groupby('sid').size()
good_items = item_sizes.index[item_sizes >= min_sc]
user_sizes = df.groupby('uid').size()
good_users = user_sizes.index[user_sizes >= min_uc]

In [33]:
item_sizes.head()

sid
0558925278    2
0733001998    1
0737104473    1
0762451459    1
1304139212    1
dtype: int64

In [34]:
good_items[:5]

Index(['0558925278', '130414643X', '1304168522', '1304174778', '1304174867'], dtype='object', name='sid')

In [35]:
# 굳이 반복문을 사용한 이유는 모르겠지만 어쨌든 개수가 적은 아이템, 유저 항목을 제거하는 작업은 맞다.
while len(good_items) < len(item_sizes) or len(good_users) < len(user_sizes):
        if min_sc > 1:
            item_sizes = df.groupby('sid').size()
            good_items = item_sizes.index[item_sizes >= min_sc]
            df = df[df['sid'].isin(good_items)]

        if min_uc > 1:
            user_sizes = df.groupby('uid').size()
            good_users = user_sizes.index[user_sizes >= min_uc]
            df = df[df['uid'].isin(good_users)]

        item_sizes = df.groupby('sid').size()
        good_items = item_sizes.index[item_sizes >= min_sc]
        user_sizes = df.groupby('uid').size()
        good_users = user_sizes.index[user_sizes >= min_uc]

In [36]:
# 정수 값 밀집화
def densify_index(df):
    print('Densifying index')
    umap = {u: i for i, u in enumerate(set(df['uid']), start=1)}
    smap = {s: i for i, s in enumerate(set(df['sid']), start=1)}
    df['uid'] = df['uid'].map(umap)
    df['sid'] = df['sid'].map(smap)
    return df, umap, smap

# train, val, test 데이터 분리
def split_df(df, user_count):
    print('Splitting')
    user_group = df.groupby('uid')
    user2items = user_group.apply(
        lambda d: list(d.sort_values(by=['timestamp', 'sid'])['sid']))
    train, val, test = {}, {}, {}
    for i in range(user_count):
        user = i + 1
        items = user2items[user]
        train[user], val[user], test[user] = items[:-2], items[-2:-1], items[-1:]
    return train, val, test

In [37]:
df, umap, smap = densify_index(df)
train, val, test = split_df(df, len(umap))

Densifying index
Splitting


C:\Users\sinny\AppData\Local\Temp\ipykernel_6292\4138822388.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user2items = user_group.apply(


In [38]:
meta = {smap[k]: v for k, v in meta_dict.items() if k in smap}

In [39]:
# 5개 출력
for i in list(meta.keys())[:5]:
    print(f"{i} : '{meta[i]}'")

29791 : 'Goodskin Labs Eyliplex-2 Eye Life and Circle Reducer - 10ml'
76720 : '*Macely* Good Afternoon Apple Cinnamon TEA Bb 30ml/1fl.zo Spa 20 Pa++ / Bb Sunblock Cream'
610 : '*Macely* Good Afternoon Honey Black Tea -- Elastic Durable Spf20 Pa+'
60236 : 'NEW EUROPEAN COLLECTION Too Faced Natural Eyes Shadow Palette 9 Colors'
1127 : '*Macely* Good Afternoon Strawberry Berry Tea --Bb 30ml/1fl.zo Spa 20 Pa++ / Bb Sunblock Cream Anti-wrinkle Nourishing'


In [40]:
dataset = {'train': train,
           'val': val,
           'test': test,
           'meta': meta,
           'umap': umap,
           'smap': smap}


In [41]:
for i in list(train.keys())[:5]:
    print(f"{i} : '{train[i]}'")

1 : '[]'
2 : '[47885, 24253, 52995, 57980, 18128, 44304, 87449, 96812, 2334, 66906, 2277, 6237, 11185, 15289, 21005, 36338, 49757, 53768, 69562, 71939, 84800]'
3 : '[58290, 348]'
4 : '[]'
5 : '[9172]'


In [42]:
for i in list(val.keys())[:5]:
    print(f"{i} : '{val[i]}'")

1 : '[50897]'
2 : '[85412]'
3 : '[12314]'
4 : '[17753]'
5 : '[78381]'


In [43]:
for i in list(test.keys())[:5]:
    print(f"{i} : '{test[i]}'")

1 : '[26560]'
2 : '[96465]'
3 : '[38771]'
4 : '[37205]'
5 : '[41711]'


In [44]:
for i in list(meta.keys())[:5]:
    print(f"{i} : '{meta[i]}'")

29791 : 'Goodskin Labs Eyliplex-2 Eye Life and Circle Reducer - 10ml'
76720 : '*Macely* Good Afternoon Apple Cinnamon TEA Bb 30ml/1fl.zo Spa 20 Pa++ / Bb Sunblock Cream'
610 : '*Macely* Good Afternoon Honey Black Tea -- Elastic Durable Spf20 Pa+'
60236 : 'NEW EUROPEAN COLLECTION Too Faced Natural Eyes Shadow Palette 9 Colors'
1127 : '*Macely* Good Afternoon Strawberry Berry Tea --Bb 30ml/1fl.zo Spa 20 Pa++ / Bb Sunblock Cream Anti-wrinkle Nourishing'


In [45]:
for i in list(umap.keys())[:5]:
    print(f"{i} : '{umap[i]}'")

A2F0MX3FT66GL1 : '1'
A3617VA1S2M2JC : '2'
A255PUDQVC0RHX : '3'
A3TALRD4RL85VG : '4'
A2ERWAPVG5DFBJ : '5'


In [46]:
for i in list(smap.keys())[:5]:
    print(f"{i} : '{smap[i]}'")

B0020MMBS0 : '1'
B00E5DPP62 : '2'
B001VHGI0W : '3'
B002RT715E : '4'
B005OZFV50 : '5'
